In [7]:
import time
from PIL import Image
import numpy as np
from naoqi import ALProxy

Navigation = ALProxy("ALNavigation", "192.168.18.60", 9559)
Posture = ALProxy("ALRobotPosture", "192.168.18.60", 9559)
Motion = ALProxy("ALMotion", "192.168.18.60", 9559)

In [9]:
#Crea la mappa
def Mapping():
    Radius = 10 #Choose the radius you want to use for the exploration
    time.sleep(10)

    #Motion.wakeUp()  #Wakes up the robot, if it's awaken does nothing
    Motion.setOrthogonalSecurityDistance(0.25) #These two lines sets the security distances os the robot
    Motion.setTangentialSecurityDistance(0.04)
    
    error = Navigation.explore(Radius)
    if error != 0:
        print ("Mapping failed")
    else:
        print ("Mapping done")
    time.sleep(2)
    
    path = Navigation.saveExploration() #saves the previous exploration
    print(path) #prints the path of the file which contains the map previously created
    
    Navigation.startLocalization() 
    Navigation.navigateToInMap([0, 0, 0]) #Returns to the starting point. The robot not always goes to the exact same point as it started because of variations. But the right origin position is where you started the exploration.
    Navigation.stopLocalization()
    Posture.goToPosture("StandInit", 0.5) 

    Final_map = Navigation.getMetricalMap()
    data = Final_map[4] #gets the data of the map 
    size = Final_map[1] #size of the map
    size2 = Final_map[2] #size of the map
    img = np.array(data).reshape(size, size2)
    img = (100 - img) * 2.5   #shapes the image to fit the map
    img = np.array(img, np.uint8)
    NewMap = Image.frombuffer('L',  (size, size2), img, 'raw', 'L', 0, 1)
    NewMap.show() #Shows the map on the computer
    #/home/nao/.local/share/Explorer/2023-06-30T131753.642Z.explo

Mapping()

Mapping done
/home/nao/.local/share/Explorer/2023-06-30T131753.642Z.explo


In [11]:
#Permette di prendere le coordinate sulla mappa
class Places_Function():  
    def Places_Config(self):
        Navigation.stopLocalization()
        #self.path = "/home/nao/.local/share/Explorer/2019-07-09T083744.926Z.explo" #Paste the path of the map here
        self.path = "/home/nao/.local/share/Explorer/2023-06-30T131753.642Z.explo" #NOSTRO
        
        Motion.setOrthogonalSecurityDistance(0.3)
        Motion.setTangentialSecurityDistance(0.06)
        Loading = Navigation.loadExploration(self.path)
        print(Loading) #True if successful
    
        Navigation.getMetricalMap()
        Navigation.startLocalization()
        Navigation.relocalizeInMap([0, 0, 0]) #start at home position (defined by the user)
        print("Relocalize done")
        
    def Places_Coordinates(self): #Place the robot in the right position and then run this
        print(Navigation.getRobotPositionInMap()) #get the position of the robot in the map
        Posture.goToPosture("StandInit", 0.5)
        Navigation.stopLocalization()
        print("Coordinates done")

In [15]:
#Rilocalizza il robot
places = Places_Function()
places.Places_Config()

True
Relocalize done


In [20]:
#Stampio le cordinate di nostro interesse
places.Places_Coordinates()

[[6.65988826751709, 6.709805011749268, -0.45311856269836426], [0.08870594948530197, 0.13751870393753052, 0.0909709557890892]]
Coordinates done


punto start [[-0.6652528643608093, 0.05108293518424034, 0.7967304587364197], [0.06300114840269089, 0.050942499190568924, 0.10306835919618607]]
punto cestini [[3.7158021926879883, 4.714118003845215, 2.3866002559661865], [0.05512908473610878, 0.09336268901824951, 0.06947634369134903]]
Infondo corridoio dritto [[6.65988826751709, 6.709805011749268, -0.45311856269836426], [0.08870594948530197, 0.13751870393753052, 0.0909709557890892]]   

In [2]:
#da eseguire per fare una foto con Pepper
import numpy as np
import cv2
from naoqi import ALProxy

def take_picture():
    # Indirizzo IP e porta del robot Pepper
    ip = "192.168.34.60"
    port = 9559

    # Creazione di un'istanza del proxy per il modulo di visione
    camProxy = ALProxy("ALVideoDevice", ip, port)

    print("Risoluzione " , camProxy.getResolution(0))
    # Configurazione dei parametri per l'acquisizione dell'immagine
    resolution = 3  # 640x480 pixels
    colorSpace = 11  # RGB color space
    fps = 30  # 30 frames per second
    cameraID = 0  # ID della fotocamera principale

    # Avvio della visualizzazione della fotocamera
    videoClient = camProxy.subscribeCamera(
        "python_client", cameraID, resolution, colorSpace, fps
    )

    # Acquisizione di un frame dalla fotocamera
    result = camProxy.getImageRemote(videoClient)

    # Convertire i dati dell'immagine in un array numpy
    imageWidth = result[0]
    imageHeight = result[1]
    imageData = np.frombuffer(result[6], dtype=np.uint8).reshape((imageHeight, imageWidth, 3))

    # Convertire BGR in RGB
    image_rgb = cv2.cvtColor(imageData, cv2.COLOR_BGR2RGB)

    # Percorso in cui salvare l'immagine
    imagePath = "C:\\Users\\luigi\\Documents\\personeDaRiconoscere\\foto.jpg"

    # Salvataggio dell'immagine su disco
    cv2.imwrite(imagePath, image_rgb)

    # Stop della visualizzazione della fotocamera
    camProxy.unsubscribe(videoClient)




In [ ]:
#fai la foto
take_picture()

In [3]:
#con nicola
import time
from PIL import Image
import numpy as np
from naoqi import ALProxy


class Navigation:

    def __init__(self):
        point_cestini = [3.715, 4.714, 2.386]
        point_corridoio = [6.65, 6.7, -0.4]
        home = [0, 0, 0]
        self.points = {}
        self.points["Corridoio"] = point_corridoio
        self.points["Home"] = home
        self.points["Cestini"] = point_cestini
        self.robot_IP = "192.168.34.60"
        self.port = 9559
        self.path = "/home/nao/.local/share/Explorer/2023-06-30T131753.642Z.explo"

    def config_navig(self):

        self.Navigation = ALProxy("ALNavigation", self.robot_IP, self.port)
        self.Posture = ALProxy("ALRobotPosture", self.robot_IP, self.port)
        self.Motion = ALProxy("ALMotion", self.robot_IP, self.port)
        self.Localization = ALProxy("ALLocalization", self.robot_IP, self.port)

        self.Navigation.stopLocalization()  # Sets the languange and the words that should be recognized by the robot

        self.Motion.setOrthogonalSecurityDistance(0.20)  # Sets the security distance for the sensors
        self.Motion.setTangentialSecurityDistance(0.03)

        self.Motion.wakeUp()
        self.Navigation.loadExploration(self.path)  # loads the map
        self.Navigation.getMetricalMap()
        self.Navigation.startLocalization()
        self.Navigation.relocalizeInMap([0, 0, 0])
        self.Posture.goToPosture("StandInit", 0.5)

    def return_home(self):
        self.navigate_map("Home")

    def navigate_map(self, name_point):
        if name_point in self.points :
            self.Navigation.navigateToInMap(self.points[name_point])
            print("Punto Raggiunto!")
        else :
            print("Errore, punto specificato non esiste")


In [9]:
#Da far partire per iniziare il lavoro svolto
import json
import time
import naoqi
import os
from naoqi import ALProxy

IP="192.168.34.60"
tts = ALProxy("ALTextToSpeech",IP,9559)
navig = Navigation()
navig.config_navig()

last_name=None
last_date=None
last_person=os.path.getmtime('C:\\Users\\luigi\\Documents\\output.json')
while True:
# Leggi il contenuto del file JSON
  with open('C:/Users/luigi/data.json', 'r') as file:
    data = json.load(file)
# Estrai il valore di newest_name
  newest_name = data['newest_name']
  newest_date = data["newest_time"]


  if newest_date!=last_date:
    if last_name is not None:
        print(newest_name)
        tts.say("Sto andando da "+str(newest_name))
        navig.navigate_map("Cestini")
        print("fine")
        tts.say("Ciao, ci sei "+str(newest_name))
        while newest_person==last_person:
            time.sleep(3)
            take_picture()
            print("ho scattato la foto")
            print("sto aspettando che cambia immagine")
            time.sleep(7)
            print("sono passati 10 secondi")
            newest_person = os.path.getmtime('C:\\Users\\luigi\\Documents\\output.json')
        # mettere foto nella cartella foto nella cartella
        # attende se vien modificato il contenuto di un file
        # dice se il paziente è presente o no
        #ritorna alla base dopo 3 minuiti
    last_name=newest_name
    last_date=newest_date
  newest_person = os.path.getmtime('C:\\Users\\luigi\\Documents\\output.json')
  if newest_person!=last_person:
      print("primo if")
      last_person = newest_person
      with open('C:\\Users\\luigi\\Documents\\output.json', 'r') as file:
        person = file.read()
        tts.say("ciao "+str(person))
        navig.return_home()

        tts.say("il paziente "+str(person)+" è presente")
        #tts.say("Forza napoli")
        #time.sleep(3)
        #tts.say("scherzo, forza Milan hahahahahaha")

        print("Il pasiente è presente")
  elif last_person is not None:
    if time.time()-last_person >180:
      pass #Fai robe belle
      
  time.sleep(1)


Nicola
Punto Raggiunto!
fine
('Risoluzione ', 4)
ho scattato la foto
sto aspettando che cambia immagine
sono passati 10 secondi
('Risoluzione ', 4)
ho scattato la foto
sto aspettando che cambia immagine
sono passati 10 secondi
('Risoluzione ', 4)
ho scattato la foto
sto aspettando che cambia immagine
sono passati 10 secondi
primo if
Punto Raggiunto!
Il pasiente è presente
